In [1]:
import ipfshttpclient
import requests
import json 
import fiona 
import geopandas
import pandas as pd
client = ipfshttpclient.connect()
import sqlite3
import hashlib
import Crypto
from Crypto.Cipher import AES
from base64 import b64encode, b64decode
import numpy as np
import time 

ConnectionError: ConnectionError: HTTPConnectionPool(host='localhost', port=5001): Max retries exceeded with url: /api/v0/version?stream-channels=true (Caused by NewConnectionError('<ipfshttpclient.requests_wrapper.HTTPConnection object at 0x7fa0aa59c8b0>: Failed to establish a new connection: [Errno 61] Connection refused'))

# Variables:

In [ ]:
DB_Name = "SQlite-file"
username = "Auditor"
organisation = "Org1"
key = "secret-key"

In [ ]:
t = add_user(username, organisation)
tables_filtered = addDB(DB_Name)

# Upload database:

In [ ]:
upload_database(DB_Name, tables_filtered, organisation, t, key )

In [ ]:
#To load the dataframe

def get_tables(db):  #Get a list of the tables that are inside the database
    cursor = db.cursor()
    tables = list()
    for name in cursor.execute('SELECT * FROM gpkg_contents;'):
        tables.append(name[0])   
    cursor.close()
    return(tables)

def filter_tables(db, tables): #Filter for usefull tables, return a list of the useful tables. 
    tables_filterd = list()
    cursor = db.cursor()
    for table in tables:
        colNames = list() 
        for name in cursor.execute('PRAGMA table_info('+ table +');'):
            colNames.append(name[1])
        if "metahistoryinsbatchid" in colNames: #We only take tables that have the list metahistoryinsbatchid
            tables_filterd.append(table)        
    cursor.close() 
    return(tables_filterd) 


def get_amount_batches(tables_filtered, db):  #Get the numer of batched in the first col (BETTER TO LOOK AT AL DB"s AND SELECT the most)
    df = pd.read_sql_query("SELECT * from " + tables_filtered[0], db)
    return(df["metahistoryinsbatchid"].unique())
    

def addDB(File_name): #Add DB To the system 
    db = sqlite3.connect(File_name)
    tables = get_tables(db)
    tables_filtered = filter_tables(db, tables)
    print("There are: " + str(len(tables_filtered)) + " tables")
    print("Containing: "+ str(len(get_amount_batches(tables_filtered, db))) + " batches")
    db.close()
    return(tables_filtered)

#To add the dataframe

def add_user(username, organization): #add user using the API in hyperledger, returns the token needed to add transaction tot the chain. 
    URL ="http://localhost:4000/users"
    PARAMS = {'username':username, 
              'orgName': organization} 
    r = requests.post(url = URL, json = PARAMS)   
    token = json.loads(r.text)['token'] 
    return token


In [ ]:
#upload complete database, input is the database name, what tables to upload and the transaction token. After this function the data is added to the blockchain and IPFS. 
def upload_database(file_name, tables_filtered, organisation, t, key):
    db = sqlite3.connect(file_name)
    for table_name in tables_filtered:
        table = get_table(table_name, db)
        for batch_number in table["metahistoryinsbatchid"].unique():
            subset = get_batch(table, batch_number)
            upload_data(subset, organisation, table_name, t,key)
    db.close()
    return()

#retrieve the table from the DB 
def get_table(table_name, db):            
    return(pd.read_sql_query("SELECT * from " + table_name, db))     

#retrieve the specific batch from the DB 
def get_batch(data, batch_number):
    return(data.loc[data['metahistoryinsbatchid'] == batch_number])         
        
#input is the single table batch, organisation, name of the table and token, output is the upload of the single batch to ipfs and blockchain. 
def upload_data(subset, organisation, table_name, t, key):
    r = check_if_exists(organisation, table_name, int(subset["metahistoryinsbatchid"].unique()[0]))
    if len(r) > 0:   #is needed to check if the batch already is in the database, prevents reuploading. 
        print ('Batch already in Blockchain') 
    else:
        s_v, h_v = verification_attributes(subset, organisation, table_name)
        s_v_cyper, nonce  = encrypt(s_v, key)
        h_i = sent_to_ipfs(s_v_cyper)
        s_i = identification_attributes(h_i, organisation, table_name, subset, h_v)
        sent_to_chain(s_i,t)
        sent_key(h_i, nonce, key)
    return()


#Retrieve the attributes used for the integrity verification phase.    
def verification_attributes(data, organisation, table_name):
    batch_hash = [hashlib.sha256(repr(val).encode()).hexdigest() for val in data.T.apply(lambda x: (tuple(x)), axis = 1)]
    h_v = hashlib.sha256(repr("".join(batch_hash)).encode()).hexdigest()
    file = {
        'orgName': organisation,
        'tableName': table_name, 
        'batch': int(data["metahistoryinsbatchid"].unique()[0]),
        'colN': len(data.columns),
        'batchHash':h_v,
        'timeStamp': data["metahistoryvalidfrom"].unique()[0],
        'hashes': batch_hash,
        'cols:': list(data.columns) 
        }
    return(file, h_v)

#Retrieve the attributes used for the identification of the verification attributes. 
def identification_attributes(h_i, organisation, table_name, data, h_v):
    args = [h_i, organisation, table_name, int(data["metahistoryinsbatchid"].unique()[0]) , str(h_v)]
    return(args)


def encrypt(file,key):
    cipher = AES.new(bytes(key, 'utf-8'), AES.MODE_EAX)
    nonce = cipher.nonce
    ciphertext, tag = cipher.encrypt_and_digest(json.dumps(file).encode('utf-8'))
    return(ciphertext, b64encode(nonce))


def sent_key (h_i, secret, key):
    URL = "http://localhost:4000/channels/mychannel/chaincodes/privateStorage"
    PARAMS = {"fcn": "createPrivateKeyStorage", 
              "chaincodeName": "privateStorage",
              "channelName": "mychannel",
              "peers": ["peer0.org1.example.com", "peer0.org2.example.com"],
              "args" : [""],
              "transient": "{\"keys\": {\"key\":\""+ str(h_i) +"\",\"secretKey\":\""+key+"\",\"nonce\":\" "+ str(secret) +"\"}} "}
    headers = {"Authorization": "Bearer " + t}
    r = requests.post(url = URL, json = PARAMS, headers=headers)
    return()
    
#Sending the verification attributes to the distributed file system, returns the identification hash. 
def sent_to_ipfs(file):
    with open("file.txt", "w") as f:
        f.write("")
    with open("file.txt", "ab") as f:
        f.write(file)
    res = client.add('file.txt')
    return(res['Hash'])

#sending the identification attributes with identification hash and verification hash to the blockchain. 
def sent_to_chain(args, t):
    URL = "http://localhost:4000/channels/mychannel/chaincodes/IntegrityVerification"
    PARAMS = {"fcn": "createRecord", "chaincodeName": "IntegrityVerification", "channelName": "mychannel", "args" : args}
    headers = {"Authorization": "Bearer " + t}
    r = requests.post(url = URL, json = PARAMS, headers=headers)
    return()


#Uses the identification attributes to check if the spefic batch is already added to the database. 
def check_if_exists(organisation, table_name, batch_number):
    URL = "http://localhost:5984/mychannel_$integrity_verification/_find"
    PARAMS = {"selector" : {"Organisation" : organisation, "Table_name" : table_name, "Batch_ID": str(batch_number)}}
    r = requests.post(url = URL, json = PARAMS)
    return(json.loads(r.text)['docs'])